In [1]:
a=dir(__builtin__)
b=dir(__builtins__)
print( a==b , a is b , all([ k[0] is k[1]  for k in zip(a,b) ]),sep= '\t' )

b = vars(__builtins__)
print(set(a) == set(b) == set( b.keys()) == set(k for k in b) ) # 默认行为

print(len(a),len(b))


True	False	True
True
154 154


In [2]:
# str objects in built-in module

def m(k):
    return isinstance(k[1],str)

b_vars = vars(__builtins__)

s1= dict( filter(m,b_vars.items()) )
s2 = {a:b for a,b in b_vars.items() if m((a,b))}


print( s1 == s2 ,len(s1))
print(55*'=')
for k,v in s1.items() :print(f'{k:<30}',v)

True 3
__name__                       builtins
__doc__                        Built-in functions, exceptions, and other objects.

Noteworthy: None is the `nil' object; Ellipsis represents `...' in slices.
__package__                    


In [3]:
# noncallable objects in built-in module
cc= dict(filter(lambda x : not callable(x[1]) ,b.items())) 
for k in cc.items() :print( '%s%s \t\t\t %s%s' % (k[0],type(k[0]),k[1],type(k[1])) )

print(33*'=')
    
print( any(isinstance(k,type) for k in cc.values()) )
try:
    for k in cc.values() : issubclass(k,BaseException)
except Exception as e:
    print(e)
    print('不是class的对象(isinstance(x,type) != true) 不能用issubclass进行检测 ')

__name__<class 'str'> 			 builtins<class 'str'>
__doc__<class 'str'> 			 Built-in functions, exceptions, and other objects.

Noteworthy: None is the `nil' object; Ellipsis represents `...' in slices.<class 'str'>
__package__<class 'str'> 			 <class 'str'>
__spec__<class 'str'> 			 ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>)<class '_frozen_importlib.ModuleSpec'>
None<class 'str'> 			 None<class 'NoneType'>
Ellipsis<class 'str'> 			 Ellipsis<class 'ellipsis'>
NotImplemented<class 'str'> 			 NotImplemented<class 'NotImplementedType'>
False<class 'str'> 			 False<class 'bool'>
True<class 'str'> 			 True<class 'bool'>
__debug__<class 'str'> 			 True<class 'bool'>
__IPYTHON__<class 'str'> 			 True<class 'bool'>
False
issubclass() arg 1 must be a class
不是class的对象(isinstance(x,type) != true) 不能用issubclass进行检测 


In [4]:
# callable objects in built-in module
c= dict(filter(lambda x : callable(x[1]) ,b.items())) 
print( all(map(lambda x : hasattr( x[1] ,'__call__' ) , c.items() )) )
print(len(c))

print(33*'=')

import random

key = random.choice(list(c.keys()))
val = c[key]
print( key , type(key) )
print( val , repr(val) , str(val) , type(val) )

print( val.__class__ is type(val) )
try :
    if type(val) is type(abs):
        print(val.__class__,val.__class__.__mro__)
    else:
        print(val,val.__mro__)
        print( val.__mro__[0] is val )
except Exception as e:
    print( e )



True
143
ImportWarning <class 'str'>
<class 'ImportWarning'> <class 'ImportWarning'> <class 'ImportWarning'> <class 'type'>
True
<class 'ImportWarning'> (<class 'ImportWarning'>, <class 'Warning'>, <class 'Exception'>, <class 'BaseException'>, <class 'object'>)
True


In [5]:
exp,typ,fnc,oth={},{},{},{}
def fun(item):
#     print(item)
    key,val = item
    if isinstance(val,type):
        if issubclass(val,BaseException):
#             Exception
            exp[key]=val
            return False
        else:
#             class
            typ[key]=val
            return True
    else:
        if type(val) is type(abs):
#             builtin_function_or_method
            fnc[key]=val
            return True
        else:
#             other
            oth[key]=val
            return False
            
d= dict( filter(fun,c.items()) )

print( len(typ) , '+', len(fnc), '=', len(d))
print(33*'=')

# print(oth)
for k,v in oth.items() : print(k,'\t',type(v))

27 + 42 = 69
input 	 <class 'method'>
copyright 	 <class '_sitebuiltins._Printer'>
credits 	 <class '_sitebuiltins._Printer'>
license 	 <class '_sitebuiltins._Printer'>
help 	 <class '_sitebuiltins._Helper'>
display 	 <class 'function'>
get_ipython 	 <class 'method'>


In [6]:
# cause 是 Exceptions 特有的 属性
print( all(map(lambda x : BaseException in x.__mro__ , exp.values() )) )
print( all(map(lambda x : hasattr( x ,'__cause__' ), exp.values() )) )
print( any(map(lambda x : hasattr( x ,'__cause__' ), d.values() )) )# 对照,说明'特有'

print(33*'=')

# functions 都是 builtin_function_or_method 的实例
print( all(map(lambda x : isinstance( x ,abs.__class__ ), fnc.values() )) )
print( any(map(lambda x : hasattr( x ,'__mro__' ), fnc.values() )) )
print( all(map(lambda x : hasattr( x ,'__mro__' ), typ.values() )) )# 对照,说明不是class


True
True
False
True
False
True


In [7]:
cls_set = set(vars(str))
for k in typ.values(): cls_set= cls_set & set(vars(k))
print(cls_set)

cls_set = set(dir(str))
for k in typ.values(): cls_set= cls_set & set(dir(k))
print(cls_set - set(dir(object)))

fnc_set = set(dir(abs))
for k in fnc.values(): fnc_set= fnc_set & set(dir(k))
print(fnc_set - set(dir(object)))

print(33*'=')

print( chr.__text_signature__ )
print( '__call__' in set(vars(int)) )
print( int.__call__ )

{'__doc__'}
set()
{'__module__', '__call__', '__text_signature__', '__self__', '__qualname__', '__name__'}
($module, i, /)
False
<method-wrapper '__call__' of type object at 0x000007FEECC18D10>


In [8]:
# 危险操作
b = vars(__builtins__)
print(len(b))
# raise SystemExit
raise KeyboardInterrupt
a = input()
if a == 'danger':
    for k in range(20):# or range(99)
        b.popitem()
    print( b.popitem() )
print( _ )
print(len(b))

154


KeyboardInterrupt: 